3. Exploratory Data Analysis  
3.1 Imports & loading Data  
3.2 Explore the data  
   * a. Features scanning & evaluation  
   * b. Visualization  
   * c. PCA tranformation if needed
* 3.3 Conclusion & summary